In [38]:
import pandas as pd
uri = 'https://gist.githubusercontent.com/guilhermesilveira/4d1d4a16ccbf6ea4e0a64a38a24ec884/raw/afd05cb0c796d18f3f5a6537053ded308ba94bf7/car-prices.csv'
dados = pd.read_csv(uri)

In [39]:
dados.head()

,Unnamed: 0,mileage_per_year,model_year,price,sold
0,0,21801,2000,30941.02,yes
1,1,7843,1998,40557.96,yes
2,2,7109,2006,89627.50,no
3,3,26823,2015,95276.14,no
4,4,7935,2014,117384.68,yes


In [40]:
mapa = {
    'mileage_per_year': 'milhas_por_ano',
    'model_year': 'ano_do_modelo',
    'price': 'preco',
    'sold': 'vendido'
}

dados = dados.rename(columns = mapa)
dados.head()

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido
0,0,21801,2000,30941.02,yes
1,1,7843,1998,40557.96,yes
2,2,7109,2006,89627.50,no
3,3,26823,2015,95276.14,no
4,4,7935,2014,117384.68,yes


In [41]:
trocar = {
    'no': 0,
    'yes': 1
}
dados['vendido'] = dados['vendido'].map(trocar)
dados.head()


,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido
0,0,21801,2000,30941.02,1
1,1,7843,1998,40557.96,1
2,2,7109,2006,89627.50,0
3,3,26823,2015,95276.14,0
4,4,7935,2014,117384.68,1


In [42]:
from datetime import datetime

ano_atual = datetime.today().year
dados['idade_do_modelo'] = ano_atual - dados['ano_do_modelo']
dados.head()

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido,idade_do_modelo
0,0,21801,2000,30941.02,1,22
1,1,7843,1998,40557.96,1,24
2,2,7109,2006,89627.50,0,16
3,3,26823,2015,95276.14,0,7
4,4,7935,2014,117384.68,1,8


In [43]:
dados['km_por_ano'] = dados['milhas_por_ano'] * 1.60934
dados.head()

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido,idade_do_modelo,km_por_ano
0,0,21801,2000,30941.02,1,22,35085.22134
1,1,7843,1998,40557.96,1,24,12622.05362
2,2,7109,2006,89627.50,0,16,11440.79806
3,3,26823,2015,95276.14,0,7,43167.32682
4,4,7935,2014,117384.68,1,8,12770.11290


In [44]:
dados = dados.drop(columns = ['Unnamed: 0', 'milhas_por_ano', 'ano_do_modelo'], axis = 1)

In [45]:
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,22,35085.22134
1,40557.96,1,24,12622.05362
2,89627.50,0,16,11440.79806
3,95276.14,0,7,43167.32682
4,117384.68,1,8,12770.11290


In [46]:
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
import numpy as np

x = dados[['preco', 'idade_do_modelo', 'km_por_ano']]
y = dados['vendido']

seed = 5
np.random.seed(seed)
x_treino, x_teste, y_treino, y_teste = train_test_split(x,y, random_state = seed, test_size = 0.25, stratify=y)
#treinando o modelo
modelo = LinearSVC()
modelo.fit(x_treino, y_treino)
previsoes = modelo.predict(x_teste)
taxa_de_acerto = accuracy_score(y_teste, previsoes)

print('Taxa de acerto: {}%'.format((taxa_de_acerto).round(2)*100))

Taxa de acerto: 47.0%


C:\Users\joaop\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Essa taxa de 47% é boa? é ruim? precisamos comparar com a linha de base

In [47]:
from sklearn.dummy import DummyClassifier
dummy_stratified = DummyClassifier()
dummy_stratified.fit(x_treino, y_treino)
previsoes = dummy_stratified.predict(x_teste)
taxa_de_acerto = accuracy_score(y_teste, previsoes)

print('Taxa de acerto dummy: {:.2f}%'.format((taxa_de_acerto)*100))

Taxa de acerto dummy: 58.00%


In [48]:
dummy_frequent = DummyClassifier()
dummy_frequent.fit(x_treino, y_treino)
previsoes = dummy_frequent.predict(x_teste)
taxa_de_acerto = accuracy_score(y_teste, previsoes)

print('Taxa de acerto dummy: {:.2f}%'.format((taxa_de_acerto)*100))

Taxa de acerto dummy: 58.00%


In [49]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

x = dados[['preco', 'idade_do_modelo', 'km_por_ano']]
y = dados['vendido']

SEED = 5
np.random.seed(SEED)
raw_treino_x, raw_teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(x_treino), len(x_teste)))

scaler = StandardScaler()
scaler.fit(raw_treino_x)
treino_x = scaler.transform(raw_treino_x)
teste_x = scaler.transform(raw_teste_x)

modelo = SVC(gamma='auto')
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)
acerto = accuracy_score(teste_y, previsoes) * 100
print('Acerto: {:.2f}%'.format(acerto))

Treinaremos com 7500 elementos e testaremos com 2500 elementos
Acerto: 77.48%


In [50]:
from sklearn.tree import DecisionTreeClassifier
SEED = 5
np.random.seed(SEED)
raw_treino_x, raw_teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(x_treino), len(x_teste)))

scaler = StandardScaler()
scaler.fit(raw_treino_x)
treino_x = scaler.transform(raw_treino_x)
teste_x = scaler.transform(raw_teste_x)

modelo = DecisionTreeClassifier()
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)
acerto = accuracy_score(teste_y, previsoes) * 100
print('Acerto: {:.2f}%'.format(acerto))

Treinaremos com 7500 elementos e testaremos com 2500 elementos
Acerto: 73.20%
